# Random Forest

Here we explore the usage of RF

In [7]:
%matplotlib inline

import os
import binascii

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import roc_auc_score

For now we will be ok with 30 depth which is already too much.

In [8]:
ESTIMATOR_LEN = 120
MAX_DEPTH = 30

First we will load the data

In [46]:
features_all_df = pd.read_csv("training_set_features.csv", index_col="respondent_id")
labels_all_df = pd.read_csv("training_set_labels.csv", index_col="respondent_id")

test_set_features_df = pd.read_csv("test_set_features.csv", index_col="respondent_id")

submission_df = pd.read_csv("submission_format.csv", index_col="respondent_id")

features_raw_df, features_test_raw_df, labels_df, labels_test_df = train_test_split(
    features_all_df,
    labels_all_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_all_df,
    random_state=123321
   # random_state=31415
)

features_raw_df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21258,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,2.0,1.0,4.0,1.0,1.0,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",0.0,0.0,wxleyezf,mxkfnird
1897,3.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,5.0,4.0,4.0,5.0,5.0,1.0,18 - 34 Years,12 Years,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,atmpeygn,Non-MSA,1.0,2.0,xicduogh,qxajmpny
6692,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,4.0,2.0,4.0,5.0,4.0,4.0,35 - 44 Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,lrircsnp,Non-MSA,0.0,0.0,nduyfdeo,xtkaffoo
8299,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,5.0,2.0,1.0,5.0,4.0,1.0,65+ Years,Some College,White,Female,NaN,Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,NaN,NaN
18412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,2.0,4.0,1.0,2.0,45 - 54 Years,< 12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",0.0,0.0,nduyfdeo,hodpvpew
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16568,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,1.0,3.0,1.0,1.0,65+ Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
24650,3.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,5.0,4.0,5.0,5.0,4.0,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,2.0,NaN,NaN
4730,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,3.0,2.0,1.0,2.0,5.0,45 - 54 Years,College Graduate,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,kbazzjca,"MSA, Not Principle City",2.0,0.0,wxleyezf,emcorrxb


In [10]:
def ordinalize(features: pd.DataFrame, raw: pd.DataFrame, name: str) -> pd.DataFrame:
    ordinal_arr = OrdinalEncoder().fit_transform(raw[[name]])

    
    features.drop(name, axis=1)
    features[name] = ordinal_arr
    return features

    
def onehotize(features: pd.DataFrame, raw: pd.DataFrame, name: str) -> pd.DataFrame:
    enc = OneHotEncoder(sparse=False)
    onehot_df = pd.DataFrame(
        enc.fit_transform(raw[name]),
        columns=enc.get_feature_names_out(),
    )
    onehot_df.index = raw.index
    return pd.concat(
        [features.drop(name, axis=1), onehot_df],
        axis=1,
    )

def normalize(in_df: pd.DataFrame) -> pd.DataFrame:
    out_df = in_df
    out_df = onehotize(out_df, in_df, ["employment_industry", "employment_occupation", "race", "hhs_geo_region", "census_msa"])
    out_df = ordinalize(out_df, in_df, "age_group")
    out_df = ordinalize(out_df, in_df, "education")
    out_df = ordinalize(out_df, in_df, "sex")
    out_df = ordinalize(out_df, in_df, "income_poverty")
    out_df = ordinalize(out_df, in_df, "marital_status")
    out_df = ordinalize(out_df, in_df, "rent_or_own")
    out_df = ordinalize(out_df, in_df, "employment_status")
    return out_df.replace(np.nan, -1)


features_df = normalize(features_raw_df)
features_test_df = normalize(features_test_raw_df)

features_df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,hhs_geo_region_fpwskwrf,hhs_geo_region_kbazzjca,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
respondent_id,,,,,,,,,,,,,,,,,,,,,
21258,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1897,3.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6692,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8299,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
18412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16568,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
24650,3.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4730,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
regress_multirf = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=ESTIMATOR_LEN,
        max_depth=MAX_DEPTH,
        random_state=31415
    )
)

How we will fit:

In [12]:
%%time

regress_multirf.fit(features_df, labels_df)

CPU times: user 19.5 s, sys: 16.7 ms, total: 19.5 s
Wall time: 19.5 s


MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=30,
                                                     n_estimators=120,
                                                     random_state=31415))

In [13]:
preds = regress_multirf.predict(features_test_df)

preds_df = pd.DataFrame({"h1h1_vaccine": preds[:, 0], "seasonal_vaccine": preds[:, 1],}, index=labels_test_df.index)
preds_df

,h1h1_vaccine,seasonal_vaccine
respondent_id,,
21868,0.146238,0.433333
7534,0.066667,0.108333
816,0.016667,0.000000
10966,0.041667,0.058333
7541,0.225000,0.073064
...,...,...
17173,0.033333,0.113156
17031,0.058333,0.125000
16703,0.350000,0.526944


In [14]:
roc_auc_score(labels_test_df, preds_df)

0.8454330431859055

Now we have our tests, lets train with the whole dataset:

In [ ]:
features_submission_df = normalize(features_all_df)
features_submission_df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,sex,income_poverty,marital_status,rent_or_own,employment_status,household_adults,household_children,employment_industry_arjwrbjb,employment_industry_atmlpfrs,employment_industry_cfqqtusy,employment_industry_dotnnunm,employment_industry_fcxhlnwr,employment_industry_haxffmxo,employment_industry_ldnlellj,employment_industry_mcubkhph,employment_industry_mfikgejo,employment_industry_msuufmds,employment_industry_nduyfdeo,employment_industry_phxvnwax,employment_industry_pxcmvdjn,employment_industry_qnlwzans,employment_industry_rucpziij,employment_industry_saaquncn,employment_industry_vjjrobsf,employment_industry_wlfvacwt,employment_industry_wxleyezf,employment_industry_xicduogh,employment_industry_xqicxuve,employment_industry_nan,employment_occupation_bxpfxfdn,employment_occupation_ccgxvspp,employment_occupation_cmhcxjea,employment_occupation_dcjcmpih,employment_occupation_dlvbwzss,employment_occupation_emcorrxb,employment_occupation_haliazsg,employment_occupation_hfxkjkmi,employment_occupation_hodpvpew,employment_occupation_kldqjyjy,employment_occupation_mxkfnird,employment_occupation_oijqvulv,employment_occupation_pvmttkik,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv,employment_occupation_nan,race_Black,race_Hispanic,race_Other or Multiple,race_White,hhs_geo_region_atmpeygn,hhs_geo_region_bhuqouqj,hhs_geo_region_dqpwygqj,hhs_geo_region_fpwskwrf,hhs_geo_region_kbazzjca,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,3.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,0.0,0.0,-1.0,3.0,1.0,1.0,4.0,1.0,2.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-1.0,3.0,3.0,5.0,5.0,4.0,1.0,4.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1

In [48]:
%%time

features_submission_df = normalize(features_all_df)

multirf = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=ESTIMATOR_LEN,
        max_depth=MAX_DEPTH,
        random_state=31415
    )
)

multirf.fit(features_submission_df, labels_all_df)

predictions = multirf.predict(normalize(test_set_features_df))

predictions

CPU times: user 43.3 s, sys: 26.6 ms, total: 43.3 s
Wall time: 43.5 s


array([[0.175     , 0.18333333],
       [0.        , 0.275     ],
       [0.24166667, 0.89404762],
       ...,
       [0.11703704, 0.34166667],
       [0.01680108, 0.46644737],
       [0.725     , 0.55833333]])

In [49]:
submission_name = f"submission_{binascii.hexlify(os.urandom(8))}.csv"

np.testing.assert_array_equal(normalize(test_set_features_df).index.values,  submission_df.index.values)

submission_df["h1n1_vaccine"] = predictions[:, 0]
submission_df["seasonal_vaccine"] = predictions[:, 1]

submission_df.to_csv(submission_name, index=True)

!head submission_name

head: cannot open 'submission_name' for reading: No such file or directory
